# Necessary imports

In [1]:
%pip install scikit-learn
%pip install seaborn
%pip install pandas

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import classification_report, confusion_matrix

from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

from sklearn.neighbors import KNeighborsClassifier

import pandas
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from tabulate import tabulate

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


# Definindo constantes

In [ ]:
RANDOM_SEED = 1337
N_JOBS = -1
FOLDS = 10

# Carregamento dos dados

In [2]:
from sklearn.model_selection import train_test_split
import pandas

def load_data(file_path: str, positive_label_multiplication: int = 2) -> pandas.DataFrame:
    """ Receives a file path for the dataset training, testing and validation datasets. """

    # Loading data from csv file
    df = pandas.read_csv(file_path)

    # Selecting useful features
    useful_features = [
        "Bidder_Tendency",
        "Bidding_Ratio",
        "Successive_Outbidding",
        "Last_Bidding",
        "Auction_Bids",
        "Starting_Price_Average",
        "Early_Bidding",
        "Winning_Ratio",
        "Auction_Duration",
        "Class"
    ]

    df = df[useful_features]

    # Augmenting positive label data
    positive_labels = df[df["Class"] == 1]

    dfs_to_concat = [df]
    for _ in range(positive_label_multiplication):
        dfs_to_concat.append(positive_labels)

    df = pandas.concat(dfs_to_concat)
    df = df.sample(frac=1)

    # Separating features and labels
    columns = list(df.columns)
    features = columns[:len(columns)-1]
    label = columns[len(columns)-1:]

    X = df[features]
    y = df[label]

    # Creating training, testing and validation datasets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = RANDOM_SEED)
    X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size = 0.125, random_state = RANDOM_SEED)

    return X_train, y_train, X_test, y_test, X_valid, y_valid

In [3]:
file_path = "./dataset/Shill Bidding Dataset.csv"

X_train, y_train, X_test, y_test, X_valid, y_valid = load_data(file_path)

# Declarão de função para treinamento de modelos

A ferramenta gridsearch será usada para determinar a melhor combinação de parâmetros para cada modelo.

Uma função search é definida, que recebe uma instância de modelo a ser "variado" e um dicionário de parâmetros e suas variações. 

In [4]:
def search(model, model_parameters): 
    """ """
    categorical_features = [
        "A1_Score",
        "A2_Score",
        "A3_Score",
        "A4_Score",
        "A5_Score",
        "A6_Score",
        "A7_Score",
        "A8_Score",
        "A9_Score",
        "A10_Score",
        "gender",
        "ethnicity",
        "jundice",
        "country",
        "used_app_before",
        "relation"
    ]
    categorical_transformer = Pipeline(
        steps=[
            ("encoder", OneHotEncoder(handle_unknown="ignore"))
        ]
    )

    preprocessor = ColumnTransformer(
        transformers=[
            ("cat", categorical_transformer, categorical_features)
        ]
    )

    model_pipeline = Pipeline([
        ('prep', preprocessor),
        ('clf', model)
    ])


    grid_search = GridSearchCV(
        model_pipeline,
        model_parameters,
        n_jobs=N_JOBS,
        cv=FOLDS
    )

    grid_search.fit(X_train, y_train)

    print(f"Best score after optimization: {grid_search.best_score_}")
    print("Best params:")
    for key, value in grid_search.best_params_.items():
        print(f"{key}: {value}")

    return grid_search

# Variando K-NN

In [ ]:
knn_parameters = {
    "clf__criterion": ["gini", "entropy", "log_loss"],
    "clf__splitter": ["best", "random"],
    "clf__max_depth": range(3, 50),
    "clf__min_samples_split": range(2, 10)
}

gs_knn = search(KNeighborsClassifier(), knn_parameters)

# Variando LVQ

# Variando Decision Tree

# Variando SVM

# Variando Random Forest

# Variando Rede Neural MLP

# Variando Comitê de redes neurais Artificiais

# Variando Comitê Heterogêneo